In [ ]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 35.9 MB/s eta 0:00:00


In [ ]:
import pulp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# to display all rows columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [ ]:
df = pd.read_csv('/content/PakvsAfg_2023_1st_ODI.csv')

In [ ]:
df

,Player,Country,Role,Credits,Profit
0,H Rauf,PAK,Bowler,8.5,158
1,Mujeeb-ur-Rahman,AFG,Bowler,8.5,128
2,S Khan,PAK,Allrounder,8.5,83
3,Rashid-Khan,AFG,Bowler,9.0,80
4,M Nabi,AFG,Allrounder,8.0,73
5,I Haq,PAK,Batsman,8.0,71
6,S Afridi,PAK,Bowler,9.0,64
7,N Shah,PAK,Bowler,8.0,63
8,M Rizwan,PAK,Keeper,8.0,45
9,I Ahmed,PAK,Batsman,7.0,36


In [ ]:
df = df.join(pd.get_dummies(df[['Role', 'Country']]))
df['PlayingXI'] = 1

In [ ]:
df.head()

,Player,Country,Role,Credits,Profit,Role_Allrounder,Role_Batsman,Role_Bowler,Role_Keeper,Country_AFG,Country_PAK,PlayingXI
0,H Rauf,PAK,Bowler,8.5,158,0,0,1,0,0,1,1
1,Mujeeb-ur-Rahman,AFG,Bowler,8.5,128,0,0,1,0,1,0,1
2,S Khan,PAK,Allrounder,8.5,83,1,0,0,0,0,1,1
3,Rashid-Khan,AFG,Bowler,9.0,80,0,0,1,0,1,0,1
4,M Nabi,AFG,Allrounder,8.0,73,1,0,0,0,1,0,1


In [ ]:
teams = list(df.Country.unique())
team1, team2 = teams[0], teams[1]

In [ ]:
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

In [ ]:
pickup_status

{0: pickup_status_0,
 1: pickup_status_1,
 2: pickup_status_2,
 3: pickup_status_3,
 4: pickup_status_4,
 5: pickup_status_5,
 6: pickup_status_6,
 7: pickup_status_7,
 8: pickup_status_8,
 9: pickup_status_9,
 10: pickup_status_10,
 11: pickup_status_11,
 12: pickup_status_12,
 13: pickup_status_13,
 14: pickup_status_14,
 15: pickup_status_15,
 16: pickup_status_16,
 17: pickup_status_17,
 18: pickup_status_18,
 19: pickup_status_19,
 20: pickup_status_20,
 21: pickup_status_21}

In [ ]:
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)

In [ ]:
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

In [ ]:
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

In [ ]:
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Country_{team1}"]] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Country_{team1}"]] for obj in df.index) <= 7

In [ ]:
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Country_{team2}"]] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Country_{team2}"]] for obj in df.index) <= 6

In [ ]:
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Keeper']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Keeper']] for obj in df.index) <= 4

In [ ]:
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Batsman']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Batsman']] for obj in df.index) <= 6

In [ ]:
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Bowler']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Bowler']] for obj in df.index) <= 6

In [ ]:
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Allrounder']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Allrounder']] for obj in df.index) <= 2

In [ ]:
mo.solve()

1

In [ ]:
df['pickup_status'] = 0

In [ ]:
for obj in df.index:
  if pickup_status[obj].varValue:
    df['pickup_status'][obj] = 1

In [ ]:
myTeamdf = df[df['pickup_status'] == 1].sort_values(['Role_Batsman', 'Role_Keeper', 'Role_Allrounder', 'Role_Bowler'], ascending = False).reset_index(drop = True)

In [ ]:
myTeamdf[['Player', 'Country', 'Role', 'Credits', 'Profit', 'pickup_status']]

,Player,Country,Role,Credits,Profit,pickup_status
0,R Shah,AFG,Batsman,7.5,34,1
1,F Zaman,PAK,Batsman,8.5,14,1
2,H Shahidi,AFG,Batsman,7.0,9,1
3,B Azam,PAK,Batsman,9.0,1,1
4,R Gurbaz,AFG,Keeper,8.0,18,1
5,S Khan,PAK,Allrounder,8.5,83,1
6,A Rahman,AFG,Allrounder,5.5,6,1
7,Rashid-Khan,AFG,Bowler,9.0,80,1
8,S Afridi,PAK,Bowler,9.0,64,1
9,A Omarzai,AFG,Bowler,6.5,23,1


In [ ]:
sum(myTeamdf.Credits)

84.0